In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/predict-west-nile-virus/train.csv.zip
!unzip /kaggle/input/predict-west-nile-virus/test.csv.zip
!unzip /kaggle/input/predict-west-nile-virus/weather.csv.zip
!unzip /kaggle/input/predict-west-nile-virus/spray.csv.zip
!unzip /kaggle/input/predict-west-nile-virus/sampleSubmission.csv

## 1. Import libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import timedelta

from xgboost import XGBClassifier

from sklearn.manifold import TSNE
from sklearn.preprocessing import MultiLabelBinarizer, OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

color = sns.color_palette()
sns.set_style('darkgrid')

#
[63]:
￼
# 2. Read data

In [ ]:
data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')
weather = pd.read_csv('weather.csv')
spray = pd.read_csv('spray.csv')
print(f'\n---TRAIN DATA---\nNumber of rows: {data_train.shape[0]}\nColumns: {data_train.columns.to_list()}')
print(f'\n---TEST DATA---\nNumber of rows: {data_test.shape[0]}\nColumns: {data_test.columns.to_list()}')
print(f'\n---WEATHER DATA---\nNumber of rows: {weather.shape[0]}\nColumns: {weather.columns.to_list()}')
print(f'\n---SPRAY DATA---\nNumber of rows: {spray.shape[0]}\nColumns: {spray.columns.to_list()}')

## 3. Exploratory data analysis

In [ ]:
data_train.head()

In [ ]:
data_test.head()

In [ ]:
weather.head()

In [ ]:
spray.head()

In [ ]:
cols_to_drop = []

### Handling missing values in weather data

In [ ]:
dfs = {
    'data_train': data_train,
    'data_test': data_test,
    'weather': weather,
    'spray': spray
}

for df_name in dfs.keys():
    print(df_name)
    print(f'\tNumber of nan: {dfs[df_name].isna().sum().sum()}')
    print(f'\tNumber of "M": {(dfs[df_name]=="M").sum().sum()}')

In [ ]:
# look closer at missing values in weather data
missing_weather = pd.DataFrame((weather=='M').sum(), columns=['number'])
missing_weather['percent'] = (missing_weather.number/weather.shape[0]*100).round(1)
missing_weather

In [ ]:
# drop column with no values
cols_to_drop.append('Water1')

In [ ]:
weather_temp = weather[weather['Station']==2]
(weather_temp[['Depart', 'Depth', 'SnowFall']]=='M').sum()

All missing values in columns 'Depart', 'Depth', 'SnowFall' comes from station 2.  
'Depart' - difference between observed and expected - to drop

In [ ]:
cols_to_drop.append('Depart')

for c in ['Depth', 'SnowFall']:
    display(weather.groupby('Station')[c].value_counts())

In [ ]:
cols_to_drop += ['Depth', 'SnowFall']

In [ ]:
# check whether 'Tavg' is average of max and min by looking at values of their difference
weather_temp = weather[weather.Tavg!='M']
(weather_temp.Tavg.astype('int') - (weather_temp[['Tmax', 'Tmin']].mean(axis=1)).round()).value_counts()


In [ ]:
# close enough to replace 'M' in Tavg with mean of Tmin and Tmax 
weather.Tavg = np.where(
    weather.Tavg == 'M',
    weather[['Tmax', 'Tmin']].mean(axis=1),
    weather.Tavg
).astype('int')

In [ ]:
weather[weather.WetBulb=='M']

In [ ]:
weather_temp = weather[weather.WetBulb!='M']
weather_temp[['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb']].astype('float').corr()

'WetBulb' is strongly correlated with 'DewPoint' and temperature.  
We will use monthly median to replace 'M' values.

In [ ]:
weather['MonthYear'] = weather.Date.str[:7]

def replace_with_monthly_median(df, col):
    monthly_medians = df[df[col]!='M'][['MonthYear', col]].groupby('MonthYear').agg(
        lambda x: x.astype('float').median()
    )
    df[col] = np.where(
        df[col]=='M',
        monthly_medians.loc[df['MonthYear']][col],
        df[col]
    ).astype('float')

In [ ]:
replace_with_monthly_median(weather, 'WetBulb')

Columns 'Heat' and 'Cool' - how many degrees above and below (respectively) 65F. We will drop these columns.

In [ ]:
cols_to_drop += ['Heat', 'Cool']

In [ ]:
weather.PrecipTotal.str.strip().value_counts().sort_index()

'T'(trace) represents value > 0 and < 0.01. We will replace 'T' with '0.001' and 'M' with monthly median. 

In [ ]:
weather.PrecipTotal = weather.PrecipTotal.str.strip().str.replace('T', '0.001')

replace_with_monthly_median(weather, 'PrecipTotal')

In [ ]:
for c in ['StnPressure', 'SeaLevel', 'AvgSpeed']:
    replace_with_monthly_median(weather, c)

### Format non-numerical weather columns

In [ ]:
weather.Date = pd.to_datetime(weather.Date)

In [ ]:
weather.CodeSum = weather.CodeSum.str.split(' ').apply(lambda x: [i for i in x if i!=''])
set(weather.CodeSum.sum())

4-character codes are two 2-character codes

In [ ]:
def correct_codes(code_list):
    new_list = []
    for code in code_list:
        new_list.append(code[:2])  # we remove '+' or '-'
        if len(code)==4:
            new_list.append(code[2:])
    return new_list

weather.CodeSum = weather.CodeSum.apply(correct_codes)

In [ ]:
mlb = MultiLabelBinarizer()
code_arr = mlb.fit_transform(weather.CodeSum)

for i in range(mlb.classes_.shape[0]):
    weather[mlb.classes_[i]] = code_arr[:,i]

In [ ]:
cols_to_drop += ['Sunrise', 'Sunset', 'MonthYear', 'CodeSum']

In [ ]:
weather.drop(cols_to_drop, axis=1, inplace=True)
cols_to_drop=[]

### Merge data from stations 1 and 2

In [ ]:
weather_st1 = weather[weather.Station==1].drop('Station', axis=1)
weather_st2 = weather[weather.Station==2].drop('Station', axis=1)
weather = weather_st1.merge(weather_st2, on='Date', suffixes=['_1', '_2'])

### Virus presence

In [ ]:
wnv = data_train.WnvPresent.value_counts()
print(wnv)
print(f'\nPercent of positive cases:\n{round(wnv[1]/(wnv[0]+wnv[1]),2)}')

Data are highly imbalanced. ROC_AUC metric is a good choice for assessing model performance.

In [ ]:
data_train.Date = pd.to_datetime(data_train.Date)
data_train['Year'] = data_train.Date.dt.year
data_train['Month'] = data_train.Date.dt.month

In [ ]:
display(data_train.groupby(['Year']).agg(
    positive_cases = ('WnvPresent','sum'),
    percent = ('WnvPresent',lambda x: (x.mean()*100).round(2)))
)
display(data_train.groupby(['Month']).agg(
    positive_cases = ('WnvPresent','sum'),
    percent = ('WnvPresent',lambda x: (x.mean()*100).round(2)))
)
display(data_train.groupby(['Year','Month']).agg(
    positive_cases = ('WnvPresent','sum'),
    percent = ('WnvPresent',lambda x: (x.mean()*100).round(2)))
)

In [ ]:
nmos = data_train.groupby('WnvPresent').NumMosquitos.sum()
print(f'Fraction of infected mosquitos weighted by "NumMosquitos": {(nmos[1]/(nmos[0]+nmos[1])).round(2)}')

### Virus testing days

In [ ]:
data_test.Date = pd.to_datetime(data_test.Date)

print(f'WNV test days: {set(data_train.Date.dt.dayofweek.unique()).union(set(data_test.Date.dt.dayofweek.unique()))}')

In [ ]:
weekdays = ['Mon','Tue', 'Wed', 'Thu', 'Fri']
num2day = {i:weekdays[i] for i in range(5)}  # No tests during weekends
dayofweek_cnt_train = data_train.Date.dt.dayofweek.map(num2day).value_counts().loc[weekdays]
dayofweek_cnt_test = data_test.Date.dt.dayofweek.map(num2day).value_counts().loc[weekdays]

fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].bar(dayofweek_cnt_train.index, dayofweek_cnt_train)
ax[0].set_title('train data')
ax[1].bar(dayofweek_cnt_test.index, dayofweek_cnt_test)
ax[1].set_title('test data')
plt.suptitle('Days of the week when tests were performed')
plt.show()

As it is stated in the problem description, mosquitos are catched in traps from Monday to Wednesday and are tested for the West Nile virus by the end of the week. The 'Date' column in train and test datasets contains a date of a test. We can conclude that tested mosquitos were captured during the same week and no later than Wednesday or the test date. Thus, for tests taken on Monday we will assume that mosquitos were collected on the same day. For tests taken on Tuesday we will assume that mosquitos were collected either on Monday or Tuesday. For tests taken on Wednesday, Thursday or Friday we will assume the mosquitos were collected on any day from Monday to Wednesday. This consideration may be important when deciding what time range of weather data should be included in additional features related to weather conditions.

### Species

In [ ]:
print(f'Train data species:\n{data_train.Species.value_counts()}')
print(f'\nTest data species:\n{data_test.Species.value_counts()}')

We can see that train dataset does not include all of the species present in the test set. Also, two species are very poorly represented in the training data. At the same time we can see completely different species distribution in the test dataset.

In [ ]:
species = data_train.Species.value_counts().to_frame()
species['wnv_present_cnt'] = data_train.groupby('Species').WnvPresent.sum()
species['wnv_present_pct'] = data_train.groupby('Species').WnvPresent.mean().round(4)*100
species

Positive cases are provided only for 3 most common species.

### Traps

In [ ]:
print(f'Number of traps:\n \
    data_train - {data_train.Trap.nunique()}\n \
    data_test - {data_test.Trap.nunique()}')

In [ ]:
trap_diff = set(data_test.Trap.unique())-set(data_train.Trap.unique())
print(f'Traps not present in training data:\n{trap_diff}') 

### Spray

In [ ]:
spray['Date'] = pd.to_datetime(spray.Date)
spray['Year'] = spray.Date.dt.year
spray['Month'] = spray.Date.dt.month

spray.groupby(['Year', 'Month']).size()

We want be using spray data in this project

## 4. Feature engineering

In [ ]:
data_train.columns

#### Extract date related features

In [ ]:
def split_date(df):
    df['Day'] = df['Date'].dt.day
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['Dayofweek'] = df['Date'].dt.dayofweek
    df['Dayofyear'] = df['Date'].dt.dayofyear

split_date(data_train)
split_date(data_test)

### Drop address

In [ ]:
def drop_address(df):
    df.drop(['Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy'], axis=1, inplace=True)
    
drop_address(data_train)
drop_address(data_test)

###  Handle categorical features

In [ ]:
np.hstack([data_train.Species.unique(), data_test.Species.unique()])

In [ ]:
oe_species = OrdinalEncoder()
oe_species.fit(np.hstack([data_train.Species.unique(), data_test.Species.unique()]).reshape(-1,1))
data_train['Species'] = oe_species.transform(data_train.Species.values.reshape(-1,1))
data_test['Species'] = oe_species.transform(data_test.Species.values.reshape(-1, 1))

In [ ]:
# replace satellite trap with main trap
data_train.Trap = data_train.Trap.str[:4]
data_test.Trap = data_test.Trap.str[:4]

oe_traps = OrdinalEncoder()
oe_traps.fit(np.hstack([data_train.Trap.unique(), data_test.Trap.unique()]).reshape(-1,1))
data_train['Trap'] = oe_traps.transform(data_train.Trap.values.reshape(-1,1))
data_test['Trap'] = oe_traps.transform(data_test.Trap.values.reshape(-1,1))

In [ ]:
data_train.Year = data_train.Year % 2007/2
data_test.Year = data_test.Year % 2007/2

### Aggregate weather data

Calculate moving average with window 3 (this is due to not being able to give precise date of collecting mosquitos)

In [ ]:
def aggregate_weather(df_weather, window=3):
    df_weather = df_weather.sort_values('Date').reset_index(drop=True)

    agg_cols = [c for c in df_weather.columns if c != 'Date']

    weather_agg = df_weather.rolling(window, on='Date').mean()

    weather_agg.loc[0, agg_cols] = df_weather.loc[0, agg_cols]
    for i in range(1, window-1):
        weather_agg.loc[i, agg_cols] = df_weather.loc[0:(i+1), agg_cols].mean(
            axis=0)

    return weather_agg

weather_agg = aggregate_weather(weather)

### Join virus testing data and aggregated weather data

In [ ]:
def add_max_catch_date(df):
    df['MaxCatchDate'] = np.where(
        df.Dayofweek<3,
        df.Date,
        np.where(
            df.Dayofweek==3,
            df.Date - timedelta(days=1),
            df.Date - timedelta(days=2)
        )
    )
    
add_max_catch_date(data_train)
add_max_catch_date(data_test)

In [ ]:
data_train = data_train.merge(weather_agg, left_on='MaxCatchDate', right_on='Date')
data_test = data_test.merge(weather_agg, left_on='MaxCatchDate', right_on='Date')

In [ ]:
cols_to_drop = ['Date_x', 'Date_y', 'Day', 'Dayofweek', 'MaxCatchDate']
data_train.drop(cols_to_drop+['NumMosquitos'], axis=1, inplace=True)
data_test.drop(cols_to_drop+['Id'], axis=1, inplace=True)

In [ ]:
labels = data_train.pop('WnvPresent')

In [ ]:
print(data_train.columns)
print(data_test.columns)

In [ ]:
data_test = data_test[data_train.columns]

In [ ]:
df = data_train.copy()
df['virus present'] = labels.astype('bool')
cols_to_plot = ['Trap', 'Longitude', 'Latitude', 'Dayofyear']
col_num = len(cols_to_plot)

fig, ax = plt.subplots(col_num,2, figsize=(15,20))
for i in range(col_num):
    sns.kdeplot(df[labels==0][cols_to_plot[i]], ax=ax[i,0], label='False')
    sns.kdeplot(df[labels==1][cols_to_plot[i]], ax=ax[i,0], label='True')
    ax[i,0].set_title(f'"{cols_to_plot[i]}" distibution conditioned on virus presence')
    ax[i,0].legend(title='virus present')
    sns.kdeplot(x=cols_to_plot[i], data=df, hue='virus present', ax=ax[i,1])
    ax[i,1].set_title(f'"{cols_to_plot[i]}" distibution divided into virus presence cases')

### Scale features

In [ ]:
scaler = StandardScaler()
scaler.fit(data_train)
cols = data_train.columns
data_train[cols] = scaler.transform(data_train)
data_test[cols] = scaler.transform(data_test)
    

### Visualize training data

In [ ]:
tsne = TSNE(n_components=2)
X = tsne.fit_transform(data_train)

df = pd.DataFrame()
df['tsne-one'] = X[:,0]
df['tsne-two'] = X[:,1]
df['virus presence']=labels.astype('bool')

plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-one", 
    y="tsne-two",
    data=df,
    hue="virus presence",
    palette=['blue', 'red'],
    alpha=0.7
)
plt.show()

## 5. Model training

In [ ]:
param_grid = {
    'n_estimators': [40, 50, 60],
    'max_depth': [4, 5, 6],
    'reg_alpha': [0.1, 0.5, 1],
    'reg_lambda': [1, 10],
    'gamma': [0, 0.5],
    'colsample_bytree': [0.8],
    'min_child_weight': [1, 3]
}

xgb = XGBClassifier(scale_pos_weight=20)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3)
model = GridSearchCV(
    param_grid=param_grid,
    estimator=xgb,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

model.fit(data_train, labels)
best_model = model.best_estimator_

In [ ]:
print(f'Best_model params: {model.best_params_}')
print(f'Best score: {round(model.best_score_, 4)}')

### Feature importance

In [ ]:
feature_important = best_model.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values("score")
data.plot(kind='barh', figsize = (20,10));


In [ ]:
model.best_estimator_.fit(data_train, labels)
predictions = model.best_estimator_.predict_proba(data_test[cols])[:,1]
sample = pd.read_csv('sampleSubmission.csv')
sample['WnvPresent'] = predictions
sample.to_csv('xgb_predictions2.csv', index=False)